Download the tool here:  http://www.kazusa.or.jp/komics/en/tool-en/218-flavonoidsearch.html

see also: Akimoto N, Ara T, Nakajima D, Suda K, Ikeda C, Takahashi S, Muneto R, Yamada M, Suzuki H, Shibata D and Sakurai N (2017) FlavonoidSearch: A system for comprehensive flavonoid annotation by mass spectrometry, Scientific Reports 7: 1243 (2017)

In [1]:
SampleList=[]

ScanList=[]
with open("spiked_feb_clean.mgf", 'r') as f:     #Reads a GNPS like mgf into a List  
 counter=0
 for line in f:
     line=line.strip('\n')
     if  line.startswith('FEATURE_ID'):
         ListID=line.split('=')
         ScanList.append(ListID)       
          
     elif line.startswith('PEPMASS'):
         ListPEPMASS=line.split('=')
         #ListPEPfloat= float(ListPEPMASS[1])
         ScanList.append(ListPEPMASS) 
         
     elif line.startswith('SCANS'):
         ListSCANS=line.split('=')
         ScanList.append(ListSCANS)
         
     elif line.startswith('RTINSECONDS'):
         ListRT=line.split('=')
         ScanList.append(ListRT)
         
     elif line.startswith('CHARGE'):
         ListCHARGE=line.split('=')
         #ListCHARGEfloat= float(ListCHARGE[1])
         ScanList.append(ListCHARGE)
         #ScanList.append([])
         #print ListCHARGE
     elif line.startswith('MSLEVEL'):
         ListRT=line.split('=')
         ScanList.append(ListRT)
         
     elif line.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')):
         ListMZandInt=line.split(' ')
         #print(ListMZandInt)
         #ListMZandIntfloat0= float(ListMZandInt[0])
         #ListMZandIntfloat1= float(ListMZandInt[1])
         #ListMZandIntfloat=[ListMZandIntfloat1,ListMZandIntfloat0]
         
         #print(ScanList)
         ScanList.append(ListMZandInt)
         
         #print ScanList
         #sys.exit(0)
         #print ListMZandInt
     elif line.startswith('END IONS'):
         if len(ScanList[5:])>4:   # parse only if more than three fragments
                SampleList.append(ScanList)
         ScanList=[]

In [2]:
for i in SampleList:
    with open('/Users/delser/Git/phd/files/%s.txt' %(i[0][1]) , 'w') as f: # intermediate files for each feature
        f.write("PrecursorMZ: ")
        f.write(i[1][1])
        f.write('\n')
        f.write("Num peaks: ")
        f.write(str(len(i)-6))
        f.write('\n')
        for item in i[6:]:
            f.write(item[0])
            f.write('\t')
            f.write(str(int(float(item[1])) )) #E to integer
            f.write('\n')
            
       

In [3]:
import glob
import subprocess
results=[]

for filepath in glob.iglob('/Users/delser/Git/phd/files/*.txt'): # run flavonoid tool for each feature file
    #print(filepath)
    f_id=filepath[:-4].split("/")
    #print(f_id[6])
    x=subprocess.check_output(['java', '-jar','FlavonoidSearchTool.jar', '-i', '%s' %(filepath) , '-p' ,'0.005']) # accuracy in da
    y=(x.decode('ascii'))
    output=y.split("\n")
    results.append([f_id[6],output])
   

In [4]:
with open("/Users/delser/Git/phd/files/output.txt", 'w') as f: #write everthing down into one output file
    f.write('ID\tNo\tJaccardScore\tIonizedAglyconeFormula\tIonizedAglyconeMass\tMSMS-categoryID\tMSMS-categoryName\tSymbolizedNameID\tSymbolizedName\tMSMS-aglyconeID\tMSMS-aglyconeName\tRelatedID\tRelatedCAS')
    f.write('\n')
    for item in results:
        if not item[1][0] =='No hit found.':
            #print(item[0])
            for i in item[1][1:-1]:
                f.write(item[0])
                f.write('\t')
                f.write(i)
                f.write('\n')
        else:
            pass


    